In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import os
from utils import calculate_metrics, count_parameters

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter('runs/q1/fc')

In [ ]:
# initialize the dataset and split it into train and test sets with a ratio of 70:30
# transform normalize and resize the images to 150*150
transform = transforms.Compose([transforms.Resize((150, 150)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = torchvision.datasets.ImageFolder(root='Shoe vs Sandal vs Boot Dataset', transform=transform)
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
def plot_images(num_images=1, root_dir="Shoe vs Sandal vs Boot Dataset"):
    classes = os.listdir(root_dir)
    plt.figure(figsize=(len(classes) * 2, num_images * 2))
    for i in range(len(classes)):
        for j in range(num_images):
            # get a random image from the class
            img_name = np.random.choice(os.listdir(os.path.join(root_dir, classes[i])))
            img_path = os.path.join(root_dir, classes[i], img_name)
            img = plt.imread(img_path)
            plt.subplot(num_images, len(classes), i * num_images + j + 1)
            plt.imshow(img)
            plt.title(classes[i])
            plt.axis("off")
    plt.show()
plot_images()

In [ ]:
class MLP(nn.Module):
    def __init__(self, hidden_layers):
        super(MLP, self).__init__()
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_layers) - 1):
            self.hidden_layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        for i, linear in enumerate(self.hidden_layers):
            if i != len(self.hidden_layers) - 1:
                x = F.relu(linear(x))
                x = self.dropout(x)
            else:
                x = self.softmax(linear(x))
        return x

In [ ]:
# define a function to train on the train set and return the loss and accuracy of train and test sets
# add the loss and accuracy to the tensorboard for each epoch
def train(model, optimizer, train_loader, test_loader, epochs, criterion):
    train_losses = []
    test_losses = []
    train_acc = []
    test_acc = []
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        train_correct = 0
        test_correct = 0
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            train_correct += pred.eq(labels.view_as(pred)).sum().item()
        train_losses.append(train_loss / len(train_loader))
        train_acc.append(train_correct / len(train_loader.dataset))
        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                loss = criterion(output, labels)
                test_loss += loss.item()
                pred = output.argmax(dim=1, keepdim=True)
                test_correct += pred.eq(labels.view_as(pred)).sum().item()
        test_losses.append(test_loss / len(test_loader))
        test_acc.append(test_correct / len(test_loader.dataset))
        print(f'Epoch: {epoch + 1}/{epochs}, '
                f'Train loss: {train_loss / len(train_loader):.3f}, '
                f'Train accuracy: {train_correct / len(train_loader.dataset):.3f}, '
                f'Test loss: {test_loss / len(test_loader):.3f}, '
                f'Test accuracy: {test_correct / len(test_loader.dataset):.3f}')
        
        writer.add_scalar('Loss/train', train_loss / len(train_loader), epoch)
        writer.add_scalar('Loss/test', test_loss / len(test_loader), epoch)
        writer.add_scalar('Accuracy/train', train_correct / len(train_loader.dataset), epoch)
        writer.add_scalar('Accuracy/test', test_correct / len(test_loader.dataset), epoch)
    return train_losses, test_losses, train_acc, test_acc

In [ ]:
hidden_layers = [150 * 150 * 3, 128, 64, 3]
criterion = nn.CrossEntropyLoss()
fc_model = MLP(hidden_layers).to(device)
# add the model to the tensorboard
writer.add_graph(fc_model, torch.rand(1, 3, 150, 150).to(device))
writer.close()

optimizer = optim.Adam(fc_model.parameters(), lr=1e-4)
train_loss, test_loss, train_acc, test_acc = train(fc_model, optimizer, train_loader, test_loader, epochs=10, criterion=criterion)
# save the model
torch.save(fc_model.state_dict(), "fc_model.pt")

In [ ]:
print(f'The model has {count_parameters(fc_model):,} trainable parameters')    
calculate_metrics(fc_model, test_loader, device=device)

In [ ]:
class CNN(nn.Module):
    def __init__(self, block_dropout=True):
        super(CNN, self).__init__()
        self.block_dropout = block_dropout
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 18 * 18, 400)
        self.fc2 = nn.Linear(400, 3)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.batchnorm4 = nn.BatchNorm1d(400)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.maxpool(F.relu(self.batchnorm1(self.conv1(x))))
        if self.block_dropout:
            x = F.dropout2d(x, 0.2, training=self.training)
        else:
            x = F.dropout(x, 0.2, training=self.training)
        x = self.maxpool(F.relu(self.batchnorm2(self.conv2(x))))
        if self.block_dropout:
            x = F.dropout2d(x, 0.2, training=self.training)
        else:
            x = F.dropout(x, 0.2, training=self.training)
        x = self.maxpool(F.relu(self.batchnorm3(self.conv3(x))))
        if self.block_dropout:
            x = F.dropout2d(x, 0.2, training=self.training)
        else:
            x = F.dropout(x, 0.2, training=self.training)
        x = x.view(-1, 64 * 18 * 18)
        x = F.relu(self.batchnorm4(self.fc1(x)))
        x = F.dropout(x, 0.2, training=self.training)
        x = self.softmax(self.fc2(x))
        return x

In [ ]:
# initialize the model and move it to GPU and add to tensorboard
writer = SummaryWriter("runs/q1_cnn_dropout")
cnn_model = CNN(block_dropout=False).to(device)
writer.add_graph(cnn_model, torch.rand(1, 3, 150, 150).to(device))

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=1e-4)

# train the model
cnn_train_losses, cnn_train_acc, cnn_test_losses, cnn_test_acc = train(cnn_model, optimizer, train_loader,test_loader, 10, criterion)
# save the model
torch.save(cnn_model.state_dict(), "cnn_model_droput.pt")

In [ ]:
print(f'The model has {count_parameters(cnn_model):,} trainable parameters')
calculate_metrics(cnn_model, test_loader, device=device)

In [ ]:
# initialize the model and move it to GPU and add to tensorboard
writer = SummaryWriter("runs/q1_cnn_block_dropout")
cnn_model_dropout = CNN(block_dropout=True).to(device)
writer.add_graph(cnn_model_dropout, torch.rand(1, 3, 150, 150).to(device))

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model_dropout.parameters(), lr=1e-4)

# train the model
cnn_train_losses, cnn_train_acc, cnn_test_losses, cnn_test_acc = train(cnn_model_dropout, optimizer, train_loader,test_loader, 10, criterion)
# save the model
torch.save(cnn_model_dropout.state_dict(), "cnn_model_block_dropout.pt")

In [ ]:
print(f'The model has {count_parameters(cnn_model_dropout):,} trainable parameters')
calculate_metrics(cnn_model_dropout, test_loader, device=device)